# Trading Strategy Project 002 - Data Exploration

This notebook provides initial exploration of the BTC/USDT hourly data and demonstrates the usage of the strategy modules.

In [ ]:
# Imports
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from src.data_loader import load_and_preprocess_data
from src.indicators import add_all_indicators
from src.signals import create_signals
from src.backtest import run_backtest
from src.metrics import calculate_all_metrics
from src.visualization import plot_portfolio_value, plot_returns, plot_drawdown

# Display settings
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 1. Load and Explore Data

In [ ]:
# Load data
train_data, test_data, val_data = load_and_preprocess_data('../data/Binance_BTCUSDT_1h.csv')

print(f"\nTraining data shape: {train_data.shape}")
print(f"Test data shape: {test_data.shape}")
print(f"Validation data shape: {val_data.shape}")

In [ ]:
# Display first few rows
train_data.head()

In [ ]:
# Basic statistics
train_data.describe()

In [ ]:
# Plot price over time
plt.figure(figsize=(14, 6))
plt.plot(train_data.index, train_data['close'], label='Close Price', linewidth=1)
plt.title('BTC/USDT Close Price - Training Data', fontsize=16)
plt.xlabel('Time')
plt.ylabel('Price (USDT)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 2. Add Technical Indicators

In [ ]:
# Add indicators to training data
train_with_indicators = add_all_indicators(
    train_data.copy(),
    rsi_period=14,
    macd_fast=12,
    macd_slow=26,
    macd_signal=9,
    bb_period=20,
    bb_std=2.0
)

train_with_indicators.head(30)

In [ ]:
# Visualize indicators
fig, axes = plt.subplots(4, 1, figsize=(14, 12))

# Price and Bollinger Bands
axes[0].plot(train_with_indicators.index, train_with_indicators['close'], label='Close', linewidth=1)
axes[0].plot(train_with_indicators.index, train_with_indicators['bb_upper'], label='BB Upper', linestyle='--', alpha=0.7)
axes[0].plot(train_with_indicators.index, train_with_indicators['bb_middle'], label='BB Middle', linestyle='--', alpha=0.7)
axes[0].plot(train_with_indicators.index, train_with_indicators['bb_lower'], label='BB Lower', linestyle='--', alpha=0.7)
axes[0].set_title('Price and Bollinger Bands')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# RSI
axes[1].plot(train_with_indicators.index, train_with_indicators['rsi'], label='RSI', color='purple', linewidth=1)
axes[1].axhline(y=70, color='r', linestyle='--', alpha=0.5, label='Overbought (70)')
axes[1].axhline(y=30, color='g', linestyle='--', alpha=0.5, label='Oversold (30)')
axes[1].set_title('RSI')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# MACD
axes[2].plot(train_with_indicators.index, train_with_indicators['macd'], label='MACD', linewidth=1)
axes[2].plot(train_with_indicators.index, train_with_indicators['macd_signal'], label='Signal', linewidth=1)
axes[2].bar(train_with_indicators.index, train_with_indicators['macd_histogram'], label='Histogram', alpha=0.3)
axes[2].set_title('MACD')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

# Volume
axes[3].bar(train_with_indicators.index, train_with_indicators['volume'], alpha=0.5)
axes[3].set_title('Volume')
axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Generate Trading Signals

In [ ]:
# Create signals with 2-of-3 confirmation
train_with_signals = create_signals(
    train_with_indicators,
    rsi_oversold=30,
    rsi_overbought=70,
    min_confirmations=2
)

# Count signals
print(f"Buy signals: {train_with_signals['buy_signal'].sum()}")
print(f"Sell signals: {train_with_signals['sell_signal'].sum()}")

In [ ]:
# Visualize signals on price chart
plt.figure(figsize=(14, 6))

# Plot price
plt.plot(train_with_signals.index, train_with_signals['close'], label='Close Price', linewidth=1.5)

# Plot buy signals
buy_signals = train_with_signals[train_with_signals['buy_signal']]
plt.scatter(buy_signals.index, buy_signals['close'], marker='^', color='green', s=100, label='Buy Signal', zorder=5)

# Plot sell signals
sell_signals = train_with_signals[train_with_signals['sell_signal']]
plt.scatter(sell_signals.index, sell_signals['close'], marker='v', color='red', s=100, label='Sell Signal', zorder=5)

plt.title('Trading Signals on Price Chart', fontsize=16)
plt.xlabel('Time')
plt.ylabel('Price (USDT)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Run Backtesting

In [ ]:
# Run backtest
portfolio, history_df = run_backtest(
    train_with_signals,
    initial_cash=10000.0,
    transaction_fee=0.00125,
    stop_loss=0.02,
    take_profit=0.03
)

print("\nBacktest complete!")
print(f"Initial portfolio value: ${portfolio.initial_cash:.2f}")
print(f"Final portfolio value: ${history_df['portfolio_value'].iloc[-1]:.2f}")
print(f"Total return: {((history_df['portfolio_value'].iloc[-1] / portfolio.initial_cash) - 1) * 100:.2f}%")

In [ ]:
# View portfolio history
history_df.head(20)

## 5. Calculate Performance Metrics

In [ ]:
# Calculate all metrics
metrics = calculate_all_metrics(
    portfolio_values=history_df['portfolio_value'],
    history_df=history_df,
    risk_free_rate=0.0,
    periods_per_year=8760
)

# Display metrics
print("\n" + "="*50)
print("PERFORMANCE METRICS")
print("="*50)
print(f"Total Return: {metrics['total_return_pct']:.2f}%")
print(f"Sharpe Ratio: {metrics['sharpe_ratio']:.4f}")
print(f"Sortino Ratio: {metrics['sortino_ratio']:.4f}")
print(f"Calmar Ratio: {metrics['calmar_ratio']:.4f}")
print(f"Max Drawdown: {metrics['max_drawdown_pct']:.2f}%")
print(f"Win Rate: {metrics['win_rate_pct']:.2f}%")
print(f"Total Trades: {metrics['total_trades']}")
print(f"Winning Trades: {metrics['winning_trades']}")
print(f"Losing Trades: {metrics['losing_trades']}")
print("="*50)

## 6. Visualizations

In [ ]:
# Portfolio value over time
plot_portfolio_value(history_df, title='Portfolio Value Over Time - Training Data')

In [ ]:
# Cumulative returns
plot_returns(history_df, title='Cumulative Returns - Training Data')

In [ ]:
# Drawdown analysis
plot_drawdown(history_df, title='Drawdown Analysis - Training Data')

## 7. Next Steps

- Run hyperparameter optimization using Optuna
- Evaluate on test and validation sets
- Perform walk-forward analysis
- Generate final report with all visualizations

To run the complete pipeline, execute:
```bash
python main.py
```